In [1]:
'''
This program was written by Joe Stephenson during the Sanger Scholar Program. You are free 
to use/alter it however you want. I hope that this helps you collect data for whatever purpose
you need it for!
'''
import os
import pandas as pd
from numpy import array
import numpy as np
import math
import time
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from datetime import datetime
from datetime import date
from collections import defaultdict
from iexfinance.stocks import get_historical_intraday

#example list of companies.
companies = [
        #utlilities 
            'AES', 'LNT', 'AEE', 'AEP', 'AWK',
            'CNP', 'CMS', 'ED', 'D', 'DTE',
            'DUK', 'EIX', 'ETR', 'EVRG', 'ES',
            'EXC', 'FE', 'NEE', 'NI', 'NRG',
            'PNW', 'PPL', 'PEG', 'SRE', 'SO',
            'WEC', 'XEL',
        #energy
            'APC','APA', 'BHGE', 'COG',
            'CVX', 'XEC', 'CXO', 'COP', 'DVN', 
            'EOG', 'EQT', 'XOM', 'HAL', 'HP',
            'HES', 'KMI', 'MRO', 'MPC', 'NOV',
            'NFX', 'NBL', 'OXY', 'OKE', 'PSX',
            'PXD', 'RRC', 'SLB', 'FTI', 'VLO', 'WMB', '']
MEASUREMENTS = [
                'average',
                'changeOverTime',
                'close',
                'date',
                'high',
                'low',
                'marketAverage',
                'marketChangeOverTime',
                'marketClose', 
                'marketHigh',
                'marketLow',
                'marketNotional',
                'marketNumberOfTrades',
                'marketOpen',
                'marketVolume',
                'notional',
                'open',
                'volume'
                        ]
#We want to know how many 
NUM_MEASURE = len(MEASUREMENTS)
#We create a variable to keep track of how many companies we are drawing data on
COUNT = len(companies)
#we need to know the day today
TODAY = date.today()
#we want to create a variable that stores the day furthest in the past that we can draw data on
THEN = TODAY - relativedelta(months=2)
#I think this is the furthest you can draw data on, correct me if i am wrong
THEN = date(THEN.year, THEN.month, 1)
#get the total number of days in between
TOTAL = TODAY - THEN
TOTAL = TOTAL.days
#get columns to print out what variable we can select from.
GET_COLUMNS = get_historical_intraday('LNT', THEN, output_format='pandas') 
#prints out this information
print(THEN, "to", TODAY, 'is the range of data being pulled')
print(TOTAL, "total number of days will be pulled")
print(COUNT, "total number of companies will be pulled from")
print("""
""")
print("A list of all the variables that you can pull data on")
print()
#creates a list of our column names and prints them out
columns = []
for col in GET_COLUMNS.columns: 
    #print(col) 
    columns.append(col)

    


2019-03-01 to 2019-05-08 is the range of data being pulled
68 total number of days will be pulled
58 total number of companies will be pulled from


A list of all the variables that you can pull data on



In [2]:
#we define a function, wutfinance() that we can pass our MEASUREMENTS array into
#if you want to change what variables are pulled, add or delete from this list.
def wut_iex(variable):
    #we want to go through all of our companies
    for x in companies[0:-1]:
        #define an empty array to store all of the dates that we get data on
        dates = []
        t_values = []
        #create an empty defaultdict which we will use to create lists of lists
        data_list = defaultdict(list)
        #we have to pull one day at a time with IEX finance. 
        for t in range(TOTAL):
            #we update the day that we are drawing data on
            END_DATE = THEN + timedelta(days=t)
            #print out the date that we are drawing data for
            print('Obtaining data for', x, 'for day', t+1)
            #obtain this data from the IEX finance data api
            data = get_historical_intraday(x, END_DATE, output_format='pandas')
            #if IEX returns a dataset, it will invariably have length of 390.
            #we can take advantage of this and skip days that do not return data
            if len(data) == 390: 
                #we want to keep track of what days we do draw data, this way we can label 
                #our datasets index later
                dates.append(END_DATE)
                #We create a unique lists for each variable. This will allow us to store
                #single variables on each company.
                for okay in MEASUREMENTS:
                    #convert our variable into strings so that we can use them to index
                    j = str(okay)
                    #draw the data and define it as its variable and company
                    data[x+'.'+j] = data[(j)]
                    #append this data to the corresponding list 
                    data_list[j].append(data[x+'.'+j])
            else:
                #we print that there was no data for this day. This is common. The stock market
                #is closed on weekends and holidays.
                print('no data for day', t)
                #keep track of the days that we have missed
                t_values.append(t)
        #we have finished drawing data. now we want to save it
        print('finished pulling data on', x) 
        #We save each of the variables on each company
        for i in MEASUREMENTS:
            #np.unique(dates)
            j =str(i)
            #select the correct variable corresponding to our company
            data = data_list[j]
            #convert it to a pandas dataframe. Pass in the index that we store from earlier
            data = pd.DataFrame(data, index = dates)
            #save this file to a csv
            data.to_csv(x+'.'+j+'.csv')
        #find how many days did not have data collected on them   
        missed_days = len(t_values)
        #find how many days did have data collected on them
        made_days = len(dates)
        all_days = missed_days + made_days
        if all_days != TOTAL:
            print('It looks like you missed some days')
        else:
            print('Finished saving data on', x, 'during the', all_days, 'peroid there were',
                  missed_days, 'days missed and', made_days, 'days')
t_values = []
dates = []    
wut_iex(MEASUREMENTS)